In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os

In [2]:
# Caminho para as pastas de treinamento
base_dir = 'D:\Repositorio\RPS-project\CNN\RockPaperScissor\\rps-cv-images'  # Substitua pelo caminho do seu diretório

# Geradores de dados para treinamento e validação com aumento de dados
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(200, 300),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(200, 300),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
num_train_samples = train_generator.samples
num_val_samples = validation_generator.samples
print(f'Número de arquivos usados para treinamento: {num_train_samples}')
print(f'Número de arquivos usados para validação: {num_val_samples}')
class_indices = train_generator.class_indices
print(f'Classes do treinamento: {class_indices}')


Found 2242 images belonging to 3 classes.
Found 558 images belonging to 3 classes.
Número de arquivos usados para treinamento: 2242
Número de arquivos usados para validação: 558
Classes do treinamento: {'paper': 0, 'rock': 1, 'scissors': 2}


In [3]:

# Definindo a CNN
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 300, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))  # Três classes de saída


d:\Repositorio\RPS-project\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:

# Resumo do modelo
model.summary()

# Compilando o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 298, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 149, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 147, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 103040)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    13,189,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,282,883 (50.67 MB)

 Trainable params: 13,282,883 (50.67 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:

# Treinando o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Epoch 1/10


d:\Repositorio\RPS-project\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


70/70 ━━━━━━━━━━━━━━━━━━━━ 56s 745ms/step - accuracy: 0.4137 - loss: 1.4205 - val_accuracy: 0.4651 - val_loss: 0.9852
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4688 - loss: 1.0003 - val_accuracy: 0.7857 - val_loss: 0.8615
Epoch 3/10


C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


70/70 ━━━━━━━━━━━━━━━━━━━━ 56s 767ms/step - accuracy: 0.5641 - loss: 0.9214 - val_accuracy: 0.5643 - val_loss: 0.9530
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5312 - loss: 0.9899 - val_accuracy: 0.5000 - val_loss: 0.8651
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 55s 760ms/step - accuracy: 0.6415 - loss: 0.7966 - val_accuracy: 0.7371 - val_loss: 0.6158
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 957us/step - accuracy: 0.8125 - loss: 0.4848 - val_accuracy: 0.8571 - val_loss: 0.3692
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 55s 755ms/step - accuracy: 0.7672 - loss: 0.5769 - val_accuracy: 0.7923 - val_loss: 0.5083
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 826us/step - accuracy: 0.7812 - loss: 0.5827 - val_accuracy: 0.7143 - val_loss: 0.6824
Epoch 9/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 54s 738ms/step - accuracy: 0.8045 - loss: 0.5084 - val_accuracy: 0.8309 - val_loss: 0.4518
Epoch 10/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 906us/step - accuracy: 0.8750 - loss: 0.2541 - val_accuracy: 0.7143 - v

In [ ]:

# Avaliando o modelo
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print(f'\nValidation accuracy: {test_acc}')


In [ ]:

# Plotando gráficos de precisão e perda
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.show()


In [6]:
model.save('D:\Repositorio\RPS-project\CNN\RockPaperScissor\\aug.h5')  # Substitua pelo caminho do seu diretório

In [ ]:
from PIL import Image
import numpy as np
image_path = 'D:\\Repositorio\\RPS-project\\CNN\\Notebook\\test.png'  # Substitua pelo caminho da sua imagem
img = Image.open(image_path)
img = img.resize((300, 200))
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Adiciona uma dimensão para o batch


In [ ]:

# Fazendo a previsão
predictions = model.predict(img_array)
predicted_label = np.argmax(predictions)

# Nome das classes
class_names = list(train_generator.class_indices.keys())

# Exibindo a imagem e a previsão
plt.figure()
plt.imshow(np.squeeze(img_array))
plt.title(f'Prediction: {class_names[predicted_label]}')
plt.axis('off')
plt.show()
